In [18]:
import pandas as pd
import numpy as np


In [2]:
csv_file = 'gps_data.csv'
data = pd.read_csv(csv_file)

In [12]:
i = data.loc[data['version'] == 'Yest'].index

In [16]:
data = data.drop(i)

In [21]:
leftest_topest = data.loc[data['longitude'].idxmin()]  # Leftmost
rightest_topest = data.loc[data['longitude'].idxmax()]  # Rightmost
leftest_downest = data.loc[data['latitude'].idxmax()]  # Bottommost
print("Leftmost-Topmost Point: (x:{}, y:{}, floor:{}, longitude: {}, latitude: {})".format(leftest_topest['x'],leftest_topest['y'],leftest_topest['floor'],leftest_topest['longitude'], leftest_topest['latitude']))
print("Rightmost-Topmost Point: (x:{}, y:{}, floor:{}, longitude: {}, latitude: {})".format(rightest_topest['x'],rightest_topest['y'],rightest_topest['floor'],rightest_topest['longitude'], rightest_topest['latitude']))
print("Leftmost-Bottommost Point: (x:{}, y:{}, floor:{}, longitude: {}, latitude: {})".format(leftest_downest['x'],leftest_downest['y'],leftest_downest['floor'],leftest_downest['longitude'], leftest_downest['latitude']))


Leftmost-Topmost Point: (x:175, y:316, floor:-1, longitude: 34.81756666666667, latitude: 32.113205)
Rightmost-Topmost Point: (x:1326, y:30, floor:0, longitude: 34.81849833333333, latitude: 32.11331833333334)
Leftmost-Bottommost Point: (x:557, y:308, floor:0, longitude: 34.81800333333333, latitude: 32.113953333333335)


In [26]:
leftest_topest = data.loc[data['x'].idxmin()] # Leftmost
rightest_topest = data.loc[data['x'].idxmax()] # Rightmost
leftest_downest = data.loc[data['y'].idxmax()] # Bottommost
leftest_topest[['x','y','longitude','latitude']],rightest_topest[['x','y','longitude','latitude']],leftest_downest[['x','y','longitude','latitude']]

(x                    0
 y                   35
 longitude    34.817755
 latitude     32.113448
 Name: 24, dtype: object,
 x                 1326
 y                   30
 longitude    34.818498
 latitude     32.113318
 Name: 1, dtype: object,
 x                   35
 y                 1181
 longitude    34.817658
 latitude     32.112855
 Name: 167, dtype: object)

In [39]:
lt = leftest_topest[['x','y','longitude','latitude']].to_dict()
rt  = rightest_topest[['x','y','longitude','latitude']].to_dict()
lb = leftest_downest[['x','y','longitude','latitude']].to_dict()
print(lt,rt,lb)

{'x': 0, 'y': 35, 'longitude': 34.817755000000005, 'latitude': 32.11344833333333} {'x': 1326, 'y': 30, 'longitude': 34.81849833333333, 'latitude': 32.11331833333334} {'x': 35, 'y': 1181, 'longitude': 34.817658333333334, 'latitude': 32.112855}


In [40]:


# Define the known points
# lt = {'x': 0, 'y': 35, 'longitude': 34.817755, 'latitude': 32.113448}
# rt = {'x': 1326, 'y': 30, 'longitude': 34.818498, 'latitude': 32.113318}
# lb = {'x': 35, 'y': 1181, 'longitude': 34.817658, 'latitude': 32.112855}

# Coordinates of the bounding points
points = np.array([
    [lt['x'], lt['y']],
    [rt['x'], rt['y']],
    [lb['x'], lb['y']]
])

# Longitudes and latitudes of the bounding points
longitudes = np.array([lt['longitude'], rt['longitude'], lb['longitude']])
latitudes = np.array([lt['latitude'], rt['latitude'], lb['latitude']])

# Define the point to be transformed
point_to_transform = np.array([1326, 1181])

# Calculate the transformation using linear algebra
# Add a column of ones for the affine transformation
points_ones = np.hstack([points, np.ones((points.shape[0], 1))])
transform_matrix_longitude = np.linalg.lstsq(points_ones, longitudes, rcond=None)[0]
transform_matrix_latitude = np.linalg.lstsq(points_ones, latitudes, rcond=None)[0]

# Add a 1 to the point to be transformed
point_to_transform_ones = np.hstack([point_to_transform, 1])

# Calculate the transformed coordinates
transformed_longitude = np.dot(point_to_transform_ones, transform_matrix_longitude)
transformed_latitude = np.dot(point_to_transform_ones, transform_matrix_latitude)

print("Rightmost-Bottommost Point: (longitude: {}, latitude: {})".format(transformed_longitude, transformed_latitude))


Rightmost-Bottommost Point: (longitude: 34.818381552078776, latitude: 32.11272592560912)


In [19]:
# Coordinates of the bounding points
leftest_topest_coord = np.array([leftest_topest['x'], leftest_topest['y']])
rightest_topest_coord = np.array([rightest_topest['x'], rightest_topest['y']])
leftest_downest_coord = np.array([leftest_downest['x'], leftest_downest['y']])

# Calculate the vector from leftest_topest to rightest_topest
vector_top = rightest_topest_coord - leftest_topest_coord

# Assuming the rectangular map, calculate the vector from leftest_topest to leftest_downest
vector_left = leftest_downest_coord - leftest_topest_coord

# Calculate the rightest-downest point
rightest_downest_coord = leftest_downest_coord + vector_top

print("Rightmost-Bottommost Point: (x: {}, y: {})".format(rightest_downest_coord[0], rightest_downest_coord[1]))

Rightmost-Bottommost Point: (x: 1361, y: 1176)
